In [ ]:
#Initialize libraries and set working directory

import os
import csv
import napari
import skimage
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from skimage import data, io, filters, exposure, img_as_float, util, measure
from skimage.exposure import histogram
from skimage.filters import threshold_otsu
from skimage.filters import sobel
from skimage.segmentation import watershed
from skimage.segmentation import clear_border
from skimage.feature import peak_local_max
from skimage.color import label2rgb
from skimage.morphology import reconstruction
from skimage.measure import label, regionprops

from scipy import ndimage as ndi
from scipy.ndimage import gaussian_filter

from pathlib import Path
from natsort import natsorted, ns
from csv import DictReader


dataFolder = '/path/to/raw/data/folder'

os.chdir(dataFolder)
print("Current working directory: ", os.getcwd())

imgFiles = os.listdir()
imgFiles = natsorted(imgFiles)
print(len(imgFiles), " files found!")

In [ ]:
###Analysis loop

var = 0

while var < len(imgFiles):

    currFile = imgFiles[var]
    print(currFile)
    rein = io.imread(currFile)

    #Define channels
    dapi = rein[:,:,3]
    ubtf = rein[:,:,0]
    npm = rein[:,:,1]
    eu = rein[:,:,2]


    ##Identify nuclei

    #Rescale image
    d1 = exposure.rescale_intensity(dapi)

    # Subtract background
    d2 = img_as_float(d1)
    d2 = gaussian_filter(d2, 1)
    h = 0.75
    seed = np.copy(d2)
    seed = d2 - h
    mask = d2
    dilated = reconstruction(seed, mask, method='dilation')
    d3 = d2 - dilated

    #Binarize image
    thresh = threshold_otsu(d3)
    d4 = d3 > thresh

    #Delete border components
    d5 = clear_border(d4)

    #Define nuclei
    nuclei = label(d5)

    #Measure and plot DAPI properties
    dapiProps = measure.regionprops_table(nuclei, dapi, properties=['label', 'area', 'eccentricity','mean_intensity'])
    dapiData = pd.DataFrame(dapiProps)


    ##Identify nucleoli

    #Rescale image
    n1 = exposure.rescale_intensity(npm)

    # Subtract background
    n2 = img_as_float(n1)
    n2 = gaussian_filter(n2, 1)
    h = 0.5
    seed = np.copy(n2)
    seed = n2 - h
    mask = n2
    dilated = reconstruction(seed, mask, method='dilation')
    n3 = n2 - dilated

    #Binarize image
    thresh = threshold_otsu(n3)
    n4 = n3 > thresh

    #Delete border components
    n5 = clear_border(n4)

    #Define nucleoli
    nucleoli = label(n5)


    #Measure and plot Nucleoli properties
    npmProps = measure.regionprops_table(nucleoli, npm, properties=['label', 'area', 'eccentricity','mean_intensity'])
    npmData = pd.DataFrame(npmProps)

    #Measure and plot 5EU properties
    euProps = measure.regionprops_table(nucleoli, eu, properties=['label', 'area', 'eccentricity','mean_intensity'])
    euData = pd.DataFrame(euProps)
    raus = pd.merge(npmData, euData, left_on=["label"], right_on=["label"], how='left')
    
    
    #Save data 
    with open('Nucleoli_Measurements.csv', 'a') as measfile:
        raus.to_csv(measfile, header=False)
    
    #Advance loop
    var += 1

In [ ]:
###Single image analysis


var = 0

currFile = imgFiles[var]
rein = io.imread(currFile)

#Define channels
dapi = rein[:,:,3]
ubtf = rein[:,:,0]
npm = rein[:,:,1]
eu = rein[:,:,2]


##Identify nuclei

#Rescale image
d1 = exposure.rescale_intensity(dapi)

# Subtract background
d2 = img_as_float(d1)
d2 = gaussian_filter(d2, 1)
h = 0.75
seed = np.copy(d2)
seed = d2 - h
mask = d2
dilated = reconstruction(seed, mask, method='dilation')
d3 = d2 - dilated

#Binarize image
thresh = threshold_otsu(d3)
d4 = d3 > thresh

#Delete border components
d5 = clear_border(d4)

#Define nuclei
nuclei = label(d5)

#Measure and plot DAPI properties
dapiProps = measure.regionprops_table(nuclei, dapi, properties=['label', 'area', 'eccentricity','mean_intensity'])
dapiData = pd.DataFrame(dapiProps)


##Identify nucleoli

#Rescale image
n1 = exposure.rescale_intensity(npm)

# Subtract background
n2 = img_as_float(n1)
n2 = gaussian_filter(n2, 1)
h = 0.5
seed = np.copy(n2)
seed = n2 - h
mask = n2
dilated = reconstruction(seed, mask, method='dilation')
n3 = n2 - dilated

#Binarize image
thresh = threshold_otsu(n3)
n4 = n3 > thresh

#Delete border components
n5 = clear_border(n4)

#Define nucleoli
nucleoli = label(n5)


##Summary Plot

fig, axes = plt.subplots(ncols=4, figsize=(48, 16))
ax = axes.ravel()
ax[0] = plt.subplot(1, 4, 1)
ax[1] = plt.subplot(1, 4, 2)
ax[2] = plt.subplot(1, 4, 3)
ax[3] = plt.subplot(1, 4, 4)

ax[0].imshow(dapi, cmap=plt.cm.gray)
ax[0].set_title('DAPI')
ax[0].axis('off')

ax[1].imshow(npm, cmap=plt.cm.nipy_spectral)
ax[1].set_title('NPM1')
ax[1].axis('off')

ax[2].imshow(eu, cmap=plt.cm.nipy_spectral)
ax[2].set_title('5EU')
ax[2].axis('off')

ax[3].imshow(n5, cmap=plt.cm.gray)
ax[3].set_title('Mask')
ax[3].axis('off')

plt.show()


#Measure and plot Nucleoli properties
npmProps = measure.regionprops_table(nucleoli, npm, properties=['label', 'area', 'eccentricity','mean_intensity'])
npmData = pd.DataFrame(npmProps)

#Measure and plot 5EU properties
euProps = measure.regionprops_table(nucleoli, eu, properties=['label', 'area', 'eccentricity','mean_intensity'])
euData = pd.DataFrame(euProps)

#Combine data for export
raus = pd.merge(npmData, euData, left_on=["label"], right_on=["label"], how='left')
plt.scatter(bin2["mean_intensity_x"], bin2["mean_intensity_y"])
plt.show()

print("Number of nucleoli found: ", len(raus))